In [1]:
import pandas as pd
from keras import Sequential

2024-01-24 08:51:10.867506: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-24 08:51:10.867538: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-24 08:51:10.868297: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-24 08:51:10.952561: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
titanic = pd.read_excel('titanic3.xls')
titanic.head()

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

# Data preparation

In [ ]:
titanic.drop(columns=['name', 'home.dest', 'ticket','cabin', 'embarked', 'boat','body', 'sibsp', 'parch', 'fare'], inplace=True)
titanic.isna().sum()

In [ ]:
titanic.dropna(inplace=True)
titanic

# ----------------------------------------------------------------------------

# Feature importance
survived rate and ratio

In [ ]:
# 0 = no, 1= yes
titanic.survived.value_counts()

In [ ]:
global_survived_rate = titanic.survived.mean()
round(global_survived_rate,2)

In [ ]:
titanic.groupby('sex').survived.mean()

In [ ]:
titanic.groupby('pclass').survived.mean()

In [ ]:
age_bins = [0, 18, 30, 50, 100]
age_labels = ['Enfant', 'Jeune Adulte', 'Adulte', 'Personne Âgée']
titanic['group_age'] = pd.cut(titanic['age'], bins=age_bins, labels=age_labels)

In [ ]:
titanic.groupby('group_age', observed=True).survived.mean()

Le groupe d'age a peu d'influence, mais il y en a

In [ ]:
titanic.drop('group_age', axis=1, inplace=True)

In [ ]:
titanic.groupby(['sex','pclass']).survived.mean()

In [ ]:
from IPython.display import display
global_survived_rate = titanic.survived.mean()
categorical_features = ['sex', 'pclass']
for c in categorical_features:
    print(c)
    df_group = titanic.groupby(c).survived.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_survived_rate
    df_group['risk'] = df_group['mean'] / global_survived_rate
    display(df_group)
    print("---")

Les paramètres qui ont le plus d'infulence sur la probabilité de survie:
1. Le sexe
2. La classe
3. L'age

# ----------------------------------------------------------------------------

In [ ]:
titanic['sex'] = titanic['sex'].apply(lambda x: 1 if x == 'female' else 0)

In [ ]:
data = titanic.drop(['survived'],axis=1)
target = titanic['survived']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=1)

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

# EDA

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
X_train

# Construction du rédeau de neurones

In [ ]:
model = Sequential()